In [7]:
from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd

In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'XPEV']

In [24]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jinchaochen/nltk_data...


True

In [21]:
# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
Best Dow Jones Stocks To Buy And Watch In March 2021: Apple, Disney, Microsoft, Visa ( Feb-28-21 04:00PM )
Buffetts Berkshire Buys Back $24.7B In Stock As 2020 Operating Income Drops ( 09:19AM )
Benzinga's Weekly Bulls And Bears: Airbnb, Apple, Kroger, Starbucks, Tesla And More ( 09:14AM )


Recent News Headlines for TSLA: 
Dow Jones Futures: Will Stock Market Rally Survive? Watch These Key Levels; Nio, Zoom On Tap ( Feb-28-21 04:08PM )
Benzinga's Weekly Bulls And Bears: Airbnb, Apple, Kroger, Starbucks, Tesla And More ( 09:14AM )
As Tesla, NIO, Xpeng and BYD battle for dominance in China, which electric car maker has the slickest fundraising machine? ( 04:30AM )


Recent News Headlines for XPEV: 
Xpeng To Release New P7 Sedan With Lithium Iron Phosphate Battery March 3: Report ( Feb-28-21 08:13AM )
As Tesla, NIO, Xpeng and BYD battle for dominance in China, which electric car maker has the slickest fundraising machine? ( 04:30AM )
7 Electric Vehicle 

In [22]:
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [25]:
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [26]:
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2021-02-28  04:00PM  0.000  0.769  0.231    0.6369
AAPL    2021-02-28  09:19AM  0.000  1.000  0.000    0.0000
AAPL    2021-02-28  09:14AM  0.000  1.000  0.000    0.0000
AAPL    2021-02-28  07:00AM  0.124  0.791  0.085   -0.1779
AAPL    2021-02-27  05:37PM  0.000  0.750  0.250    0.4588


              Date     Time    neg    neu    pos  compound
Ticker                                                    
TSLA    2021-02-28  04:08PM  0.000  1.000  0.000    0.0000
TSLA    2021-02-28  09:14AM  0.000  1.000  0.000    0.0000
TSLA    2021-02-28  04:30AM  0.115  0.749  0.137   -0.1280
TSLA    2021-02-27  11:00AM  0.000  0.723  0.277    0.3182
TSLA    2021-02-27  06:36AM  0.000  0.743  0.257    0.5859


              Date     Time    neg    neu    pos  compound
Ticker                                                    
XPEV    2021-02-28  08:13AM  0.000  1.000  0.000  

In [28]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)



Recent News Headlines for AAPL: 
Best Dow Jones Stocks To Buy And Watch In March 2021: Apple, Disney, Microsoft, Visa ( Feb-28-21 04:00PM )
Buffetts Berkshire Buys Back $24.7B In Stock As 2020 Operating Income Drops ( 09:19AM )
Benzinga's Weekly Bulls And Bears: Airbnb, Apple, Kroger, Starbucks, Tesla And More ( 09:14AM )


Recent News Headlines for TSLA: 
Dow Jones Futures: Will Stock Market Rally Survive? Watch These Key Levels; Nio, Zoom On Tap ( Feb-28-21 04:08PM )
Benzinga's Weekly Bulls And Bears: Airbnb, Apple, Kroger, Starbucks, Tesla And More ( 09:14AM )
As Tesla, NIO, Xpeng and BYD battle for dominance in China, which electric car maker has the slickest fundraising machine? ( 04:30AM )


Recent News Headlines for AMZN: 
Better Buy: Square Stock or Every Nasdaq Stock? ( Feb-28-21 11:22AM )
2 Reasons Square's Results Are Better Than You Think ( 09:00AM )
These 3 E-Commerce Stocks Had Record Years in 2020 -- and They're Just Getting Started ( 08:30AM )


              Date    

In [29]:
df

,Mean Sentiment
Ticker,
AMZN,0.13
AAPL,0.03
TSLA,0.00
